In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

### Statement of business problem
A restaurant company is seeking to start operations in Los Mochis City. They are specialized in seafood cuisine, and the high management wants to select the most suitable location. Their strategy is based on avoiding competition and placing their business in highly visible places like Malls, parks or sports fields. They want to find a location with a good mix between high visibility and low competition.

# Use geopy library to get the latitude and longitude values of Los Mochis

In [2]:
address = 'Los Mochis, Sinaloa'

geolocator = Nominatim(user_agent="lm_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Los Mochis are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Los Mochis are 25.7928058, -108.990188.


# Define our Foursquare credentials

In [59]:
# @hidden_cell

CLIENT_ID = 'AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR' # your Foursquare ID
CLIENT_SECRET = '5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR
CLIENT_SECRET:5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK


In [6]:
#We'll use four locations in Los Mochis to extract more than 100 answers per venue
long1=-108.942
lat1=25.8005
long2=-109.048
lat2=25.7911
long3=-108.988
lat3=25.8247
long4=-108.986
lat4=25.7544


# Get the venues in Los Mochis. 

# Table 1

In [7]:
LIMIT=100
radius=5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat1, 
    long1, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.8005,-108.942&radius=5000&limit=100'

## Send the GET request

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ccf4c896a6071212180ff9f'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ff8b7f7e4b0e8d00e3701b9-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ce941735',
         'name': 'Seafood Restaurant',
         'pluralName': 'Seafood Restaurants',
         'primary': True,
         'shortName': 'Seafood'}],
       'id': '4ff8b7f7e4b0e8d00e3701b9',
       'location': {'cc': 'MX',
        'country': 'México',
        'distance': 2635,
        'formattedAddress': ['México'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 25.80349096940428,
          'lng': -108.96809113698511}],
        'lat': 25.80349096940428,
        'lng': -108.96809113698511}

# Create a function to extract the venue category

In [9]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Put that into a Pandas dataframe

In [10]:
venues = results['response']['groups'][0]['items']
    
lm_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues =lm_venues.loc[:, filtered_columns]

# filter the category for each row
lm_venues['venue.categories'] = lm_venues.apply(get_category_type, axis=1)

# clean columns
lm_venues.columns = [col.split(".")[-1] for col in lm_venues.columns]

lm_venues.head()

,name,categories,lat,lng
0,Mariscos la memoria,Seafood Restaurant,25.803491,-108.968091
1,Ciclopista Cerro de La Memoria,Athletics & Sports,25.810077,-108.976175
2,Cerro de la Memoria (Mirador),Scenic Lookout,25.808307,-108.973012
3,Farmacia Guadalajara Las Fuentes,Pharmacy,25.803706,-108.979477
4,Tacos Punta Arena,Taco Place,25.791606,-108.980147


## Table 2

In [11]:
LIMIT=100
radius=5000
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat2, 
    long2, 
    radius, 
    LIMIT)
url2 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.7911,-109.048&radius=5000&limit=100'

In [12]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5ccf4ca4dd57972411ecbf60'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-50b81b92e4b0e31c3fea2eb8-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_soccer_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d188941735',
         'name': 'Soccer Stadium',
         'pluralName': 'Soccer Stadiums',
         'primary': True,
         'shortName': 'Soccer'}],
       'id': '50b81b92e4b0e31c3fea2eb8',
       'location': {'cc': 'MX',
        'city': 'Los Mochis',
        'country': 'México',
        'distance': 1841,
        'formattedAddress': ['Los Mochis, Sinaloa', 'México'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 25.79769122058063,
          'lng': -109.03114759602704}],
    

In [13]:
venues2 = results2['response']['groups'][0]['items']
    
lm_venues2 = json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues2 =lm_venues2.loc[:, filtered_columns]

# filter the category for each row
lm_venues2['venue.categories'] = lm_venues2.apply(get_category_type, axis=1)

# clean columns
lm_venues2.columns = [col.split(".")[-1] for col in lm_venues2.columns]

lm_venues2.head()

,name,categories,lat,lng
0,Estadio Centenario,Soccer Stadium,25.797691,-109.031148
1,birrieria mexico,Taco Place,25.806653,-109.022864
2,J&R Ribs LA COSTILLERIA,Bistro,25.792341,-109.010940
3,Country Club,Golf Course,25.792104,-109.009421
4,Kinatia,Tea Room,25.793400,-109.010284


## Table 3

In [14]:
LIMIT=100
radius=5000
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat3, 
    long3, 
    radius, 
    LIMIT)
url3 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.8247,-108.988&radius=5000&limit=100'

In [15]:
results3 = requests.get(url3).json()
results3

{'meta': {'code': 200, 'requestId': '5ccf4cb3dd579724104bcda7'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-50831e31e4b09e409c9616d2-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ce941735',
         'name': 'Seafood Restaurant',
         'pluralName': 'Seafood Restaurants',
         'primary': True,
         'shortName': 'Seafood'}],
       'id': '50831e31e4b09e409c9616d2',
       'location': {'address': 'Blvd. Rio Presidio',
        'cc': 'MX',
        'city': 'Los Mochis',
        'country': 'México',
        'distance': 764,
        'formattedAddress': ['Blvd. Rio Presidio',
         'Los Mochis, Sinaloa',
         'México'],
        'labeledLatLngs': [{'label': 'display',
          'l

In [16]:
venues3 = results3['response']['groups'][0]['items']
    
lm_venues3 = json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues3 =lm_venues3.loc[:, filtered_columns]

# filter the category for each row
lm_venues3['venue.categories'] = lm_venues3.apply(get_category_type, axis=1)

# clean columns
lm_venues3.columns = [col.split(".")[-1] for col in lm_venues3.columns]

lm_venues3.head()

,name,categories,lat,lng
0,Mariscos El Brujo,Seafood Restaurant,25.817996,-108.989641
1,Ciclopista Cerro de La Memoria,Athletics & Sports,25.810077,-108.976175
2,Plaza Sendero Los Mochis,Shopping Mall,25.814433,-108.978527
3,Tacos el compa luis,Steakhouse,25.815946,-108.991209
4,Caffenio Río Fuerte,Coffee Shop,25.806786,-108.994540


## Table 4

In [17]:
LIMIT=100
radius=5000
url4 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat4, 
    long4, 
    radius, 
    LIMIT)
url4 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AXA4RUWOTHDTINRO1UQOOKK5DISY3VHOAPZ0WUGEXGPVZ0SR&client_secret=5ZOES3U0DJZWO0BKZIWZHPR1XUJFHWQAPSMTYW4NWQXJTQXK&v=20180605&ll=25.7544,-108.986&radius=5000&limit=100'

In [18]:
results4 = requests.get(url4).json()
results4

{'meta': {'code': 200, 'requestId': '5ccf4cc2351e3d1d4e1a9dce'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-503feaa0e4b04fbe52032457-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pharmacy_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d10f951735',
         'name': 'Pharmacy',
         'pluralName': 'Pharmacies',
         'primary': True,
         'shortName': 'Pharmacy'}],
       'id': '503feaa0e4b04fbe52032457',
       'location': {'address': 'Centenario',
        'cc': 'MX',
        'city': 'Los Mochis',
        'country': 'México',
        'distance': 1896,
        'formattedAddress': ['Centenario', 'Los Mochis, Sinaloa', 'México'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 25.769183577161545,
          'lng': -108.99

In [19]:
venues4 = results4['response']['groups'][0]['items']
    
lm_venues4 = json_normalize(venues4) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
lm_venues4 =lm_venues4.loc[:, filtered_columns]

# filter the category for each row
lm_venues4['venue.categories'] = lm_venues4.apply(get_category_type, axis=1)

# clean columns
lm_venues4.columns = [col.split(".")[-1] for col in lm_venues4.columns]

lm_venues4.head()

,name,categories,lat,lng
0,Farmacia Guadalajara,Pharmacy,25.769184,-108.995404
1,cenaduria doña margarita,Mexican Restaurant,25.775803,-108.990302
2,Sopes El Panchito,Mexican Restaurant,25.776796,-108.977488
3,Panamá Restaurante & Pastelería,Restaurant,25.786442,-108.997372
4,"Tacos ""El Juan""",Food Truck,25.776644,-109.003657


## Stack four tables

In [20]:
lm_allvenues = pd.concat([lm_venues, lm_venues2,lm_venues3,lm_venues4])
lm_allvenues.index=range(len(lm_allvenues))
lm_allvenues


,name,categories,lat,lng
0,Mariscos la memoria,Seafood Restaurant,25.803491,-108.968091
1,Ciclopista Cerro de La Memoria,Athletics & Sports,25.810077,-108.976175
2,Cerro de la Memoria (Mirador),Scenic Lookout,25.808307,-108.973012
3,Farmacia Guadalajara Las Fuentes,Pharmacy,25.803706,-108.979477
4,Tacos Punta Arena,Taco Place,25.791606,-108.980147
5,Deportiva Aurelio Rodriguez,Sports Club,25.799885,-108.977939
6,Mariscos El Gordo,Seafood Restaurant,25.792254,-108.985767
7,Plaza Sendero Los Mochis,Shopping Mall,25.814433,-108.978527
8,Estadio Emilio Ibarra Almada,Baseball Field,25.802794,-108.974670
9,Restaurant Litzy-Wah,Chinese Restaurant,25.809330,-108.978179


## Find venues that contains malls or sports fields

In [21]:
lm_shopvenues=lm_allvenues[lm_allvenues['categories'].str.contains('Shopping Mall')]
lm_shopvenues.index=range(len(lm_shopvenues))
lm_baseballvenues=lm_allvenues[lm_allvenues['categories'].str.contains('Baseball Field')]
lm_baseballvenues.index=range(len(lm_baseballvenues))
lm_soccervenues=lm_allvenues[lm_allvenues['categories'].str.contains('Soccer Stadium')]
lm_soccervenues.index=range(len(lm_soccervenues))

## Count the venues

In [22]:
print('{} venues were returned by Foursquare.'.format(lm_allvenues.shape[0]))

345 venues were returned by Foursquare.


## Find the unique categories

In [23]:
print('There are {} uniques categories.'.format(len(lm_allvenues['categories'].unique())))
print(lm_allvenues['categories'].unique())

There are 66 uniques categories.
['Seafood Restaurant' 'Athletics & Sports' 'Scenic Lookout' 'Pharmacy'
 'Taco Place' 'Sports Club' 'Shopping Mall' 'Baseball Field'
 'Chinese Restaurant' 'Steakhouse' 'Mexican Restaurant' 'Burrito Place'
 'Multiplex' 'Breakfast Spot' 'Restaurant' 'Café' 'Food Truck' 'Gym'
 'Pizza Place' 'Italian Restaurant' 'Ice Cream Shop'
 'Health & Beauty Service' 'Food Court' 'Sandwich Place' 'Hot Dog Joint'
 'Fast Food Restaurant' 'Deli / Bodega' 'Bakery' 'Coffee Shop'
 'Sushi Restaurant' 'Snack Place' 'Hotel' 'Convention Center'
 'Convenience Store' 'Soccer Stadium' 'Bistro' 'Golf Course' 'Tea Room'
 'Park' 'Department Store' 'Mediterranean Restaurant' 'Dessert Shop'
 'Grocery Store' 'Health Food Store' 'Salad Place' 'Dumpling Restaurant'
 'Science Museum' 'Dive Bar' 'Bar' 'Farm' 'Japanese Restaurant' 'Theater'
 'Plaza' 'Movie Theater' 'Hardware Store' 'Supermarket' 'Market'
 'Tennis Court' 'Speakeasy' 'Gift Shop' 'Warehouse Store' 'Gourmet Shop'
 'Wine Shop' 'Spa

## Select only seafood venues

In [24]:
lm_sfvenues=lm_allvenues[lm_allvenues['categories'].str.contains('Seafood Restaurant')]
lm_sfvenues.index=range(len(lm_sfvenues))
lm_sfvenues

,name,categories,lat,lng
0,Mariscos la memoria,Seafood Restaurant,25.803491,-108.968091
1,Mariscos El Gordo,Seafood Restaurant,25.792254,-108.985767
2,Mariscos BajaMar,Seafood Restaurant,25.792535,-108.980962
3,Las Conchitas,Seafood Restaurant,25.787848,-108.979441
4,Del Mar La Mar,Seafood Restaurant,25.798771,-108.989406
5,La Charanga mariscos,Seafood Restaurant,25.788360,-108.986384
6,Los Abulones,Seafood Restaurant,25.792020,-108.981343
7,Mariscos El Tungar,Seafood Restaurant,25.792034,-108.990280
8,Mariscos La Palmera,Seafood Restaurant,25.802563,-108.983827
9,Mariscos Fátima,Seafood Restaurant,25.799948,-108.985984


## Create a map of Los Mochis with the venues superimposed

In [26]:
# create map of New York using latitude and longitude values
map_lm = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map for seafood venues
for lat, lng, name, category in zip(lm_sfvenues['lat'], lm_sfvenues['lng'], lm_sfvenues['name'], lm_sfvenues['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lm)  

# add markers to map for malls venues
for lat, lng, name, category in zip(lm_shopvenues['lat'], lm_shopvenues['lng'], lm_shopvenues['name'], lm_shopvenues['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lm) 

# add markers to map for baseball fields venues
for lat, lng, name, category in zip(lm_baseballvenues['lat'], lm_baseballvenues['lng'], lm_baseballvenues['name'], lm_baseballvenues['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lm) 
    
# add markers to map for soccer fields venues
for lat, lng, name, category in zip(lm_soccervenues['lat'], lm_soccervenues['lng'], lm_soccervenues['name'], lm_soccervenues['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lm) 

    
map_lm

## Analize data

# Step 1: looking for direct competence. 
Since Los Mochis is a coastal city, we expect the competence to be high, but rewards can also be high given the popularity of seafood cuisine.As we can see, ther is a high number of seafood restaurants in  Los Mochis, so we should be careful in selecting an appropriate location. 

## Step 2: looking for popular places
We find that there are only one baseball field and one coccer field, but also there are six malls. As we know, the stakeholders want their business to be visible for possible customers, and a way to get this is finding a popular place, like malls or sport fields, and locate the restaurant near of one of them.

## Step 3: selecting a place
By looking the distribution of seafood restaurants and popular spots in Los Mochis, we can see a promising place locaten in the middle of three 
highly popular places, and where there is only one seafood restaurant. The place is located in the middle of Emilio Ibarra Baseball Stadium,
Plaza Sendero and Plaza Las Fuentes. In this place we only find one seafood restaurant, called "Mariscos Las Cañas 2". Next, we show the location in 
the map.

In [56]:
lc=lm_sfvenues.loc[lm_sfvenues['name'] == 'Mariscos Las Cañas 2']
ld=lm_shopvenues.loc[lm_shopvenues['name'] == 'Plaza Sendero Los Mochis']
le=lm_shopvenues.loc[lm_shopvenues['name'] == 'Plaza Las Fuentes']


In [58]:
# create map of New York using latitude and longitude values
map_lm = folium.Map(location=[latitude, longitude], zoom_start=12)


# add markers to map for baseball fields venues
for lat, lng, name, category in zip(lm_baseballvenues['lat'], lm_baseballvenues['lng'], lm_baseballvenues['name'], lm_baseballvenues['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lm) 
    
# add markers to map for two malls venues
for lat, lng, name, category in zip(ld['lat'], ld['lng'], ld['name'], ld['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lm) 

for lat, lng, name, category in zip(le['lat'], le['lng'], le['name'], le['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lm) 
    
map_lm

### Conclusion
The zone we reccommend to start a new seafood restaurant in Los Mochis is located among two malls and one baseball field, which will allow us to be 
highly visible for possible customers, and since there in only one bussines that is our competence, is our reasoning that we can catch a significative portion of the market. 
